In [13]:
import spacy
from spacy.tokens import Span
from spacy.matcher import PhraseMatcher

nlp = spacy.load("en_core_web_sm")
doc = nlp("I love coffee")

print(doc.text)
animals = ["Golden Retriever", "cat", "turtle", "Rattus norvegicus"]
matcher = PhraseMatcher(nlp.vocab)

I love coffee


In [2]:
def animal_component(doc):
    animals = ["Golden Retriever", "cat", "turtle", "Rattus norvegicus"]
    matcher = PhraseMatcher(nlp.vocab)
    
    matches = matcher(doc)
    span = [Span(doc, start, end, label="ANIMAL") for match_id, start, end in matches]
    doc.ents = span
    return doc

In [7]:
global nlp
nlp.add_pipe(animal_component, after=False)

time_spent = []

In [49]:
from time import time

def timer(func):
    def f(*args, **kwargs):
        global time_spent
        before = time()
        rv = func(*args, **kwargs)
        after = time()
        time_spent[func.__name__] = format(after - before, '.2f')# round(int(after - before), 3)
        return rv
    return f

In [50]:
@timer
def get_entities(doc):
    return [(ent.text, ent.label_) for ent in doc.ents]

@timer
def get_verbs(doc):
    return [(token.text, token.pos_) for token in doc if token.pos_ == "VERB" or token.pos_ == "AUX"]

@timer
def get_countries(doc):
    return [(ent.text, ent.label_) for ent in doc.ents if ent.label_ == 'GPE']

@timer
def get_persons(doc):
    return [(ent.text, ent.label_) for ent in doc.ents if ent.label_ == 'PERSON']


In [51]:
TEXT = """
I dont't wanna do it today, maybe tomorrow morning.
"""
doc = nlp(TEXT)


print('VERBS   :',  get_verbs(doc))
print('VERBS   :',  get_entities(doc))
print('VERBS   :',  get_countries(doc))
print('VERBS   :',  get_persons(doc))
print(time_spent)

VERBS   : [("dont't", 'VERB'), ('do', 'AUX')]
VERBS   : []
VERBS   : []
VERBS   : []
{'get_verbs': '0.00', 'get_entities': '0.00', 'get_countries': '0.00', 'get_persons': '0.00'}


In [29]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

In [15]:
results = classifier(["We are very happy to show you the 🤗 Transformers library", "We hope you don't hate it."])

for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

label: POSITIVE, with score: 0.9998
label: NEGATIVE, with score: 0.5309


In [27]:
import csv

p = '../dataset/datasets_73978_166769_fb_sentiment.csv'
tweets = []
with open(p, newline='') as f:
    spamreader = csv.DictReader(f)
    for row in spamreader:
        tweets.append(row["FBPost"])
        
tweets[0]

'Drug Runners and  a U.S. Senator have something to do with the Murder http://www.amazon.com/Circumstantial-Evidence-Getting-Florida-Bozarth-ebook/dp/B004FPZ452/ref=pd_rhf_p_t_1 The State Attorney Knows... NOW So Will You. GET Ypur Copy TODAY'

In [30]:
doc = nlp(tweets[1])
print(doc)
print('Entities:   ', get_entities(doc))
print('Countries   :',  get_countries(doc))
print('PERSON   :',  get_persons(doc))

results = classifier(" you arent missing a thing, I tried this game and it sucks! ")
print("RESULT:  ", results)

Heres a single, to add, to Kindle. Just read this 19th century story: "The Ghost of Round Island". Its about a man (French/American Indian) and his dog sled transporting a woman across the ice, from Mackinac Island to Cheboygan - and the ghost that...
Entities:    [('this 19th century', 'DATE'), ('The Ghost of Round Island', 'WORK_OF_ART'), ('French', 'NORP'), ('Mackinac Island', 'GPE'), ('Cheboygan', 'GPE')]
Countries   : [('Mackinac Island', 'GPE'), ('Cheboygan', 'GPE')]
PERSON   : []
RESULT:   [{'label': 'NEGATIVE', 'score': 0.9923949837684631}]


In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")

In [14]:
from transformers import pipeline

context = r"""
Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a
question answering dataset is the SQuAD dataset, which is entirely based on that task. If you would like to fine-tune
a model on a SQuAD task, you may leverage the examples/question-answering/run_squad.py script.
"""

nlp = pipeline("question-answering")
result = nlp(question="What is extractive question answering?", context=context)
print(result)

/home/yanick/anaconda3/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,


{'score': 0.6225793957710266, 'start': 34, 'end': 96, 'answer': 'the task of extracting an answer from a text given a question.'}


In [15]:
nlp = pipeline("ner")

In [3]:
from transformers import pipeline

summarizer = pipeline("summarization")

In [31]:
ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

s = summarizer(ARTICLE, max_length=118, min_length=30, do_sample=False)
print(s[0]["summary_text"])

 Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002 . At one time, she was married to eight men at once, prosecutors say .


In [10]:
ARTICLE = """
In order to analyze what has been done in Cape Verde with regard to Intellectual Property 
and also Digital Signature, we interviewed the competent entities related to each particular subject
and conducted interviews and applied questionnaires that we sent through electronic mails 
and social networks to our artists and authors both in Cape Verde and some who are living
abroad. With the data collected it was possible to verify that the protection of Intellectual
Property is still in a phase of maturation in our archipelago and having been given more
attention in recent years.
"""
print(len(ARTICLE))

print(summarizer(ARTICLE, max_length=118, min_length=30, do_sample=False))

584
[{'summary_text': ' In order to analyze what has been done in Cape Verde with regard to Intellectual Property and Digital Signature, we interviewed the competent entities related to each particular subject. We conducted interviews and applied questionnaires that we sent through electronic mails and social networks to our artists and authors .'}]


In [25]:
import string
s = "@Maria: Do you mean the Nook? Be careful, books you buy for the Kindle are for that piece of electronics, and vice versa. I love my Kindle, there are people that swear by the Nook. They like the color screen.Me? I want an ereader that is a reader-- so I dont need color. The kindle battery lasts longer, and the unit isnt as heavy, which can make a difference after reading for a few hours. :) " # Sample string 
out = s.translate(str.maketrans('', '', string.punctuation))


doc = nlp(s.replace("@", ""))
print()
print('Entities:   ', get_entities(doc))
print('Countries   :',  get_countries(doc))
print('PERSON   :',  get_persons(doc))



Entities:    [('Maria', 'PERSON'), ('Kindle', 'PERSON'), ('a few hours', 'TIME')]
Countries   : []
PERSON   : [('Maria', 'PERSON'), ('Kindle', 'PERSON')]


In [3]:
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
nlp = English()

tokenizer = Tokenizer(nlp.vocab)
tokens = tokenizer("This is a sentence")
print(tokens)

This is a sentence
